# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2023spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/15lGUmT8NpLGtoxRllRWCJyQEjhR1Idcei63YHsDckPE/edit#slide=id.g21fff4e9af6_0_13)　Kaggle: [Link](https://www.kaggle.com/competitions/ml2023spring-hw7/host/sandbox-submissions)　Data: [Link](https://drive.google.com/file/d/1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7/view?usp=sharing)




# Prerequisites

## Download Dataset

In [1]:
# download dataset by youself!
# unzip dataset
!unzip -o hw7_data.zip

unzip:  cannot find or open hw7_data.zip, hw7_data.zip.zip or hw7_data.zip.ZIP.


## Install packages

Documentation for the toolkit:　https://huggingface.co/transformers/

In [2]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.26.1
!pip install accelerate==0.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


# Kaggle (Fine-tuning)

## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Import Packages

In [3]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(2)

## Load Model and Tokenizer




 

In [4]:
from transformers import (
  BertTokenizerFast,
  AutoModelForQuestionAnswering,
)

from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("luhua/chinese_pretrain_mrc_roberta_wwm_ext_large")
model = AutoModelForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_roberta_wwm_ext_large")

#model = AutoModelForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
#tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26918 QA pairs
- Dev set: 2863  QA pairs
- Test set: 3524  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [5]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

# Change the path of the dataset
train_questions, train_paragraphs = read_data("/kaggle/input/2023-ml-hw7-question-answering/hw7_train.json")
dev_questions, dev_paragraphs = read_data("/kaggle/input/2023-ml-hw7-question-answering/hw7_dev.json")
test_questions, test_paragraphs = read_data("/kaggle/input/2023-ml-hw7-question-answering/hw7_test.json")

## Tokenize Data

In [6]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False, max_length=512)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


## Dataset

In [7]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 60
        self.max_paragraph_len = 150
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 16

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])


            # A single window is obtained by slicing the portion of paragraph containing the answer
            #mid = (answer_start_token + answer_end_token) // 2
            #tt = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            
            start_min = max(0,answer_end_token - self.max_paragraph_len+1)
            start_max = min(answer_start_token, len(tokenized_paragraph) - self.max_paragraph_len)
            start_max = max(start_min,start_max)
            paragraph_start = random.randint(start_min, start_max)
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

## Function for Evaluation

In [8]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        if start_index > end_index:
            continue
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [9]:
from accelerate import Accelerator
# hyperparameters
num_epoch = 8
validation = True
logging_step = 100
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
train_batch_size = 8

#### TODO: gradient_accumulation (optional)####
# Note: train_batch_size * gradient_accumulation_steps = effective batch size
# If CUDA out of memory, you can lower the train_batch_size and upper the gradient_accumulation_steps
# Doc: https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation
gradient_accumulation_steps = 16

# dataloader
# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)


# Change "fp16_training" to True to support automatic mixed 
# precision training (fp16)	
fp16_training = True
if fp16_training:    
    accelerator = Accelerator(mixed_precision="fp16")
else:
    accelerator = Accelerator()

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/
model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

model.train()


print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])
        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
           
        train_loss += output.loss
        
        accelerator.backward(output.loss)
        
        step += 1
        optimizer.step()
        optimizer.zero_grad()
        
        ##### TODO: Apply linear learning rate decay #####
        learning_rate = learning_rate * (1.0/(1.0 + 0.00001*step))

        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Start Training ...


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 0.803, acc = 0.657
Epoch 1 | Step 200 | loss = 0.692, acc = 0.711
Epoch 1 | Step 300 | loss = 0.628, acc = 0.762
Epoch 1 | Step 400 | loss = 0.570, acc = 0.762
Epoch 1 | Step 500 | loss = 0.608, acc = 0.752
Epoch 1 | Step 600 | loss = 0.656, acc = 0.744
Epoch 1 | Step 700 | loss = 0.613, acc = 0.762
Epoch 1 | Step 800 | loss = 0.661, acc = 0.748
Epoch 1 | Step 900 | loss = 0.638, acc = 0.744
Epoch 1 | Step 1000 | loss = 0.544, acc = 0.780
Epoch 1 | Step 1100 | loss = 0.555, acc = 0.780
Epoch 1 | Step 1200 | loss = 0.602, acc = 0.761
Epoch 1 | Step 1300 | loss = 0.577, acc = 0.770
Epoch 1 | Step 1400 | loss = 0.571, acc = 0.766
Epoch 1 | Step 1500 | loss = 0.645, acc = 0.712
Epoch 1 | Step 1600 | loss = 0.514, acc = 0.771
Epoch 1 | Step 1700 | loss = 0.560, acc = 0.775
Epoch 1 | Step 1800 | loss = 0.523, acc = 0.780
Epoch 1 | Step 1900 | loss = 0.552, acc = 0.770
Epoch 1 | Step 2000 | loss = 0.666, acc = 0.736
Epoch 1 | Step 2100 | loss = 0.540, acc = 0.780
E

  0%|          | 0/2863 [00:00<?, ?it/s]

...

Validation | Epoch 7 | acc = 0.916


  0%|          | 0/3365 [00:00<?, ?it/s]

Epoch 8 | Step 100 | loss = 0.053, acc = 0.965
Epoch 8 | Step 200 | loss = 0.059, acc = 0.969
Epoch 8 | Step 300 | loss = 0.069, acc = 0.972
Epoch 8 | Step 400 | loss = 0.040, acc = 0.976
Epoch 8 | Step 500 | loss = 0.047, acc = 0.976
Epoch 8 | Step 600 | loss = 0.056, acc = 0.974
Epoch 8 | Step 700 | loss = 0.072, acc = 0.965
Epoch 8 | Step 800 | loss = 0.054, acc = 0.975
Epoch 8 | Step 900 | loss = 0.050, acc = 0.975
Epoch 8 | Step 1000 | loss = 0.041, acc = 0.984
Epoch 8 | Step 1100 | loss = 0.051, acc = 0.979
Epoch 8 | Step 1200 | loss = 0.053, acc = 0.979
Epoch 8 | Step 1300 | loss = 0.041, acc = 0.980
Epoch 8 | Step 1400 | loss = 0.065, acc = 0.969
Epoch 8 | Step 1500 | loss = 0.058, acc = 0.971
Epoch 8 | Step 1600 | loss = 0.052, acc = 0.969
Epoch 8 | Step 1700 | loss = 0.079, acc = 0.965
Epoch 8 | Step 1800 | loss = 0.060, acc = 0.969
Epoch 8 | Step 1900 | loss = 0.060, acc = 0.979
Epoch 8 | Step 2000 | loss = 0.062, acc = 0.972
Epoch 8 | Step 2100 | loss = 0.063, acc = 0.970
E

  0%|          | 0/2863 [00:00<?, ?it/s]

Validation | Epoch 8 | acc = 0.918
Saving Model ...


## Testing

In [10]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
    # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

Completed! Result is in result.csv


# GradeScope - Question 2 (In-context learning)

### Try in-context learning
The example prompt is :
```
請從最後一篇的文章中找出最後一個問題的答案：
文章：<文章內容>
問題：<問題敘述>
答案：<答案>
```

In [11]:
import torch

# To avoid CUDA_OUT_OF_MEMORY
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(2)

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# You can try model with different size
# When using Colab or Kaggle, model with more than 2 billions parameters may 
# run out of memory
tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-1.7B")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   4 # When using Colab or Kaggle, model with more than 2 billions parameters may                 │
│   5 # run out of memory                                                                          │
│   6 tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")                              │
│ ❱ 7 model = AutoModelForCausalLM.from_pretrained("facebook/xglm-1.7B")                           │
│   8                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/transformers/models/auto/auto_factory.py:465 in           │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   462 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   463 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│   464 │   │   │   return model_class.from_pretrained(                                            │
│ ❱ 465 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   466 │   │   │   )                                                                              │
│   467 │   │   raise ValueError(                                                                  │
│   468 │   │   │   f"Unrecognized configuration class {config.__class__} for this kind of AutoM   │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/transformers/modeling_utils.py:2362 in from_pretrained    │
│                                                                                                  │
│   2359 │   │   │   init_contexts.append(init_empty_weights())                                    │
│   2360 │   │                                                                                     │
│   2361 │   │   with ContextManagers(init_contexts):                                              │
│ ❱ 2362 │   │   │   model = cls(config, *model_args, **model_kwargs)                              │
│   2363 │   │                                                                                     │
│   2364 │   │   # Check first if we are `from_pt`                                                 │
│   2365 │   │   if use_keep_in_fp32_modules:                                                      │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/transformers/models/xglm/modeling_xglm.py:836 in __init__ │
│                                                                                                  │
│   833 │   def __init__(self, config):                                                            │
│   834 │   │   super().__init__(config)                                                           │
│   835 │   │   self.model = XGLMModel(config)                                                     │
│ ❱ 836 │   │   self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)        │
│   837 │   │                                                                                      │
│   838 │   │   # Initialize weights and apply final processing                                    │
│   839 │   │   self.post_init()                                                                   │
│                                                            

In [ ]:
# for cleaning model output. If you try different prompts, you may have to fix 
# this function on your own
def clean_text(text):
    # Note: When you use unilingual model, the colon may become fullwidth
    text = text.split("答案:")[-1]
    text = text.split(" ")[0]
    return text

In [ ]:
import random
import json
# Change the path of you data
with open("/kaggle/input/2023-ml-hw7-question-answering/hw7_in-context-learning-examples.json", "r") as f: 
    test = json.load(f)

# K-shot learning 
# Give model K examples to make it achieve better accuracy 
# Note: When K >= 4, CUDA_OUT_OFF_MEMORY may occur
K = 1


question_ids = [qa["id"] for qa in test["questions"]]

with torch.no_grad():
    for idx, qa in enumerate(test["questions"]):
        # You can try different prompts
        prompt = "請從最後一篇的文章中找出最後一個問題的答案\n"
        exist_question_indexs = [question_ids.index(qa["id"])]
        # K-shot learning: give the model K examples with answers
        for i in range(K):
            question_index = question_ids.index(qa["id"])
            while(question_index in exist_question_indexs): 
                question_index = random.randint(0, len(question_ids) - 1)
            exist_question_indexs.append(question_index)    
            paragraph_id = test["questions"][question_index]["paragraph_id"]
            prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
            prompt += f'問題：{test["questions"][question_index]["question_text"]}\n'
            prompt += f'答案：{test["questions"][question_index]["answer_text"]}\n'


        # The final one question without answer
        paragraph_id = qa["paragraph_id"]
        prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
        prompt += f'問題：{qa["question_text"]}\n'
        prompt += f'答案：'
        
        inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt") 
        sample = model.generate(**inputs, max_new_tokens = 20)
        text = tokenizer.decode(sample[0], skip_special_tokens=True)
        # Note: You can delete this line to see what will happen
        text = clean_text(text)
        
        print(prompt)
        print(f'正確答案: {qa["answer_text"]}')
        print(f'模型輸出: {text}')
        print()
